In [1]:
! pip install plyfile
! pip install easydict

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [21]:
! pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6988 sha256=235ef31045dd19149c95a1ab3bc90fc5e9aa3a67c09d9b050bef1b0e4c2ac15b
  Stored in directory: /users/fangjuny/.cache/pip/wheels/ab/a5/e8/c9fa20742edbccf2702dae8ee62053e6c460e961d45967b49c
Successfully built clip


In [22]:
class CLIPTextEncoder(nn.Module):
    def __init__(self, **kwargs) -> None:
        super().__init__()
        emb_dim = kwargs.get("embed_dim")#768
        self.token_embedding, self.clip_txt, self.transformer, self.ln_final, self.text_projection = self.clip_txt_transformer()

    def clip_txt_transformer(self, freeze=True):
    
        device = "cuda" if torch.cuda.is_available() else "cpu"
        clip_model_name = 'ViT-B/32'
        clip_model = clip.load(clip_model_name, device=device)[0]

        clip_txt = clip_model.transformer
        clip_txt.float()
        if freeze:
            print("------------------- Txt frozen ---------------------")
            clip_txt.eval()
            for k, v in clip_txt.named_parameters():
                v.requires_grad = False
                print(k)
            print("------------------- Txtfrozen ---------------------")
            #transformer's input output are the same(sequence length, 1, feature dimension), 
            #class_embedding
        return clip_txt.token_embedding, clip_txt.positional_embedding, clip_txt.txt.transformer, clip_txt.ln_final, clip_txt.text_projection

    def encode_text(self, text, apply_projection=True):
        x = self.token_embedding(text).type(self.dtype)
        x = x + self.positional_embedding.type(self.dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.ln_final(x).type(self.dtype)

        if apply_projection:
            # 只在需要时应用投影层
            x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection
        else:
            # 返回没有应用投影层的特征
            x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)]

        return x

    def forward(self, text, apply_text_projection=True):
        text_features = self.encode_text(text, apply_projection=apply_text_projection)
        return text_features


NameError: name 'nn' is not defined

In [2]:
! pip install SharedArray

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from __future__ import print_function
import random
import shutil
import os
import glob
from copy import deepcopy
import open3d as o3d
import torch # why is it located here?
import numpy as np
from plyfile import PlyData, PlyElement
import pdb
import cv2
cv2.setNumThreads(0)
import pytorch_lightning as pl
from pytorch_lightning.strategies import DDPStrategy
import yaml
from easydict import EasyDict
from model import get as get_model
from configs.configs import Config_train
# from model import get as get_model
from dataset import get as get_dataset
import torch
from torch.utils.data import Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed=0
pl.seed_everything(seed) # , workers=True
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) # if use multi-GPU

def read_txt(path):
    with open(path) as f:
        lines = f.readlines()
    lines = [int(x.strip()) for x in lines]
    return lines

#parser = my_args()
args = Config_train()

    # ------------
    # randomness or seed
    # ------------
torch.backends.cudnn.benchmark = args.cudnn_benchmark



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[rank: 0] Seed set to 0


In [2]:
scene_path = "/scratch/project_2002051/junyuan/cvpr24-challenge/data/ScanNet200/train"
data_ROOT = "/scratch/project_2002051/junyuan/cvpr24-challenge/data/ScanNet200"



In [3]:
from dataset.utils import transform as t

In [4]:
with open('/scratch/project_2002051/junyuan/cvpr24-challenge/data/ScanNet200/train/scene0691_00.ply', 'rb') as f:
    plydata = PlyData.read(f)
    print(plydata['vertex'])
    vertex = plydata['vertex']
    
    #coordinates
    coord = np.vstack([vertex['x'], vertex['y'], vertex['z']]).T
    print(coord.shape)
    # Extract colors
    feat = np.vstack([vertex['red'], vertex['green'], vertex['blue']]).T
        
    # Extract labels
    label = vertex['label']

    # Extract instant_id
    instant_id = vertex['instance_id']

element vertex 104270
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
property uint label
property uint instance_id
(104270, 3)


In [5]:
from dataset.utils.data_util import data_prepare_v101 as data_prepare

In [16]:
# for this dataloade, the data augmentation for s3dis dataset can be directly used in scannet dataset
# the original scale of RGB in scannet dataset is [0, 255]
# after augmentation and pre-processing, the scale of RGB will be converted into [0, 1]

# the scale of coordinate keeps the same and only has [coord_min] shift  

class Loader(Dataset):
    def __init__(self, scene_path, query_path = None, mask_path = None, mode = 'train'):
        """
        初始化点云数据集
        :param ply_files: PLY文件的列表
        """
        self.mode = mode
        self.scene_path = "/scratch/project_2002051/junyuan/cvpr24-challenge/data/ScanNet200/train"
        self.query_path = query_path
        self.mask_path = mask_path
        
        self.voxel_size = 0.04
        self.test_area = 5
        self.classes = None

        self.shuffle_index = True
        self.voxel_max = 8000000
        
        self.transform = t.Compose(
                [
                    t.RandomScale([0.9, 1.1]), 
                    t.ChromaticAutoContrast(),
                    t.ChromaticTranslation(), 
                    t.ChromaticJitter(), 
                    t.HueSaturationTranslation()
                ])
        
        self.ply_list = []  # 先初始化列表
        self.query_list = []
        self.mask_list = []
        self.scene_id = None
        
        # 然后加载数据
        self.load_ply_list()
        #self.load_query_list()
        #self.load_mask_list()
    
    def load_ply_list(self):
        """
        加载PLY文件列表
        """
        # 读取PLY文件列表
        self.ply_list = sorted(glob.glob(os.path.join(self.scene_path, '*.ply')))
        #self.data_idx = np.arange(len(self.ply_list))
        
        #print("Loaded PLY files:", self.ply_list)
    def load_query_list(self):
        if self.query_path is not None:
            # 加载CSV文件
            df = pd.read_csv(self.query_path)

            # 按照某个列的值进行排序
            df_sorted = df.sort_values(by='video_id')

            # 获取排序后的另一个列的值
            self.query_list = df_sorted['query'].values
            # # 输出获取的值
            # print(self.query_list)
            
    def load_mask_list(self):
        if self.mask_path is not None:
            self.mask_list = sorted(glob.glob(os.path.join(self.mask_path, '*.txt')))
            #print(self.mask_list)
    def __len__(self):
        """
        数据集中的样本数
        """
        return len(self.ply_list)

    def __getitem__(self, idx):
        plydata = PlyData.read(self.ply_list[idx])
        vertex_data = plydata['vertex']

        #coordinates
        coord = np.vstack([vertex['x'], vertex['y'], vertex['z']]).T
        # Extract colors
        feat = np.vstack([vertex['red'], vertex['green'], vertex['blue']]).T
        # Extract labels
        label = vertex['label']
        # Extract instant_id
        instance_id = vertex['instance_id']
        # Optionally, convert labels to a numpy array for easier manipulation
        labels_array = label.astype(np.int64)
        coord, feat, label = data_prepare(
            coord, feat, labels_array, 
            self.mode, self.voxel_size, self.voxel_max, 
            self.transform, self.shuffle_index)
        
        return coord, feat, label, instance_id

In [ ]:
class DatasetA(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        self.file_list = sorted(glob.glob(os.path.join(self.path, '*.ply')))
        self.load_files()

    def load_files(self):
        self.data = []
        for file in self.file_list:
            plydata = PlyData.read(file)
            vertex_data = plydata['vertex']
            coord = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T
            feat = np.vstack([vertex_data['red'], vertex_data['green'], vertex_data['blue']]).T
            label = vertex_data['label'].astype(np.int64)
            instance_id = vertex_data['instance_id']
            self.data.append((coord, feat, label, instance_id))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        coord, feat, label, instance_id = self.data[idx]
        if self.transform:
            coord, feat, label = self.transform(coord, feat, label)
        return coord, feat, label, instance_id


class DatasetB(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        self.file_list = sorted(glob.glob(os.path.join(self.path, '*.ply')))
        self.load_files()

    def load_files(self):
        self.data = []
        for file in self.file_list:
            plydata = PlyData.read(file)
            vertex_data = plydata['vertex']
            coord = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T
            feat = np.vstack([vertex_data['red'], vertex_data['green'], vertex_data['blue']]).T
            label = vertex_data['label'].astype(np.int64)
            instance_id = vertex_data['instance_id']
            self.data.append((coord, feat, label, instance_id))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        coord, feat, label, instance_id = self.data[idx]
        if self.transform:
            coord, feat, label = self.transform(coord, feat, label)
        return coord, feat, label, instance_id


In [17]:
scannetloader = Loader(scene_path=None)

data_iterator = iter(scannetloader)

# 获取第一个元素
first_batch = next(data_iterator)

In [22]:
first_batch[2]

#torch.Size([104270])

tensor([0, 3, 4,  ..., 3, 3, 0])

In [24]:
def TrainValCollateFn(batch):
    coord, feat, label, _ = list(zip(*batch))
    offset, count = [], 0
    for item in coord: # len of pc
        count += item.shape[0]
        offset.append(count)
    data_dict = \
        {
            'coord': torch.cat(coord),
            'feat': torch.cat(feat),
            'target': torch.cat(label),
            'offset': torch.IntTensor(offset),
        }
    return data_dict

train_loader_kwargs = \
        {
            "batch_size": 4,
            "num_workers": 2,
            "collate_fn": TrainValCollateFn,
            "pin_memory": True,
            "drop_last": False,
            "shuffle": True,
        }
train_loader = torch.utils.data.DataLoader(
        scannetloader, **train_loader_kwargs)

In [25]:
train_loader.batch_sampler()

TypeError: 'BatchSampler' object is not callable

In [9]:
# load check point for the model
from importlib import import_module
args.load_model ="/home/fangj1/Code/Vision-Language-on-3D-Scene-Understanding/EPCL/indoor_segmentation/checkpoints/epoch=062--mIoU_val=0.6972--.ckpt"
args.on_train = False
print('ckpt best. args.load_model=[{}]'.format(args.load_model))
assert args.load_model is not None, 'why did you come?'
print(args.transdown)
model = get_model(args.model).load_from_checkpoint(
    os.path.join(args.MYCHECKPOINT, args.load_model), 
    args=args).to(device) # args.strict_load

model.eval()
model.freeze()

[rank: 0] Seed set to 0


ckpt best. args.load_model=[/home/fangj1/Code/Vision-Language-on-3D-Scene-Understanding/EPCL/indoor_segmentation/checkpoints/epoch=062--mIoU_val=0.6972--.ckpt]
SymmetricTransitionDownBlock


FileNotFoundError: [Errno 2] No such file or directory: '/home/fangj1/Code/Vision-Language-on-3D-Scene-Understanding/EPCL/indoor_segmentation/checkpoints/epoch=062--mIoU_val=0.6972--.ckpt'

# Process arkit data
in original model need (n, 3), (n, c) 

In [ ]:

#data loader
from torch.utils.data import Dataset
import pandas as pd
from torch.utils.data import DataLoader
class ArkitDataLoader(Dataset):
    def __init__(self, scene_path , query_path = None, mask_path = None):
        """
        初始化点云数据集
        :param ply_files: PLY文件的列表
        """
        self.scene_path = scene_path
        self.query_path = query_path
        self.mask_path = mask_path
        self.ply_list = []  # 先初始化列表
        self.query_list = []
        self.mask_list = []
        self.scene_id = None
        
        # 然后加载数据
        self.load_ply_list()
        self.load_query_list()
        self.load_mask_list()
        
    def load_ply_list(self):
        """
        加载PLY文件列表
        """
        # 读取PLY文件列表
        self.ply_list = sorted(glob.glob(os.path.join(self.scene_path, '4*/*.ply')))
        #print("Loaded PLY files:", self.ply_list)
    def load_query_list(self):
        if self.query_path is not None:
            # 加载CSV文件
            df = pd.read_csv(self.query_path)

            # 按照某个列的值进行排序
            df_sorted = df.sort_values(by='video_id')

            # 获取排序后的另一个列的值
            self.query_list = df_sorted['query'].values
            # # 输出获取的值
            # print(self.query_list)
            
    def load_mask_list(self):
        if self.mask_path is not None:
            self.mask_list = sorted(glob.glob(os.path.join(self.mask_path, '*.txt')))
            #print(self.mask_list)
    def __len__(self):
        """
        数据集中的样本数
        """
        return len(self.ply_list)

    def __getitem__(self, idx):
        """
        读取单个点云文件，并返回其数据
        :param idx: 索引
        """
        self.scene_id = self.mask_list[idx].split('/')[-1].split('_')[0]
        #print(self.ply_list[idx].split('/')[-1].split('_')[0])
        # 加载点云文件
        pcd = o3d.io.read_point_cloud(self.ply_list[idx])

        # 获取坐标
        coordinates = np.asarray(pcd.points, dtype=np.float32)

        # 获取特征，这里假设使用颜色作为特征
        if pcd.colors:
            features = np.asarray(pcd.colors, dtype=np.float32)  # RGB颜色
        else:
            features = np.zeros((coordinates.shape[0], 3), dtype=np.float32)  # 如果没有颜色，使用零填充

        
        #mask
        if self.mask_path is not None:
            mask = read_txt(self.mask_list[idx])
        # 将数据转换为torch tensors
        mask = torch.tensor(mask).unsqueeze(1)
        coordinates = torch.from_numpy(coordinates)
        features = torch.from_numpy(features)

        #return {'coord': coordinates, 'feat': features, 'prompt': self.query_list[idx], 'target': mask}
        return coordinates, features, self.query_list[idx], mask
# def collate_fn(batch):
#     max_points = max([sample['coord'].shape[0] for sample in batch])
#     feature_dim = batch[0]['feat'].shape[1]

#     batch_coordinates = torch.zeros(len(batch), max_points, 3, device=device)
#     batch_features = torch.zeros(len(batch), max_points, feature_dim, device=device)
#     batch_prompts = []
#     batch_masks = torch.zeros(len(batch), max_points, 1,device=device)

#     for i, sample in enumerate(batch):
#         num_points = sample['coord'].shape[0]
#         batch_coordinates[i, :num_points] = sample['coord'].to(device)
#         batch_features[i, :num_points] = sample['feat'].to(device)
#         batch_prompts.append(sample['prompt'])
#         batch_masks[i, :num_points] = sample['target'].to(device)
    
#     offset, count = [], 0
#     for item in max_points: # len of pc
#         count += item.shape[0]
#         offset.append(count)   
#     return {
#         'coord': batch_coordinates,
#         'feat': batch_features,
#         'offset': torch.IntTensor(batch_offsets),
#         'prompt': batch_prompts,
#         'target': batch_masks
#     }
def TrainValCollateFn(batch):
    coord, feat, prompt, mask = list(zip(*batch))
    offset, count = [], 0
    for item in coord: # len of pc
        count += item.shape[0]
        offset.append(count)
    
    print("Coordinates type:", type(coord[0]))  # Check the type of the first coordinate set
    print("Features type:", type(feat[0]))      # Check the type of the first features set
    print("Mask type:", type(mask[0]))          # Check the type of the first mask
    print("Offset type:", type(offset[0]))      # Check the type of the first offset
    
    data_dict = \
        {
            'coord': torch.cat(coord).to(device),
            'feat': torch.cat(feat).to(device),
            'target': torch.cat(mask).to(device),
            'prompt': list(prompt),
            'offset': torch.IntTensor(offset).to(device),
        }
    return data_dict
dataset = ArkitDataLoader(args.arkit_train_root,args.development_query_root,args.development_mask_root)
#data_loader = DataLoader(dataset, batch_size= args.train_batch, collate_fn=TrainValCollateFn)
data_loader = DataLoader(dataset, batch_size= 1, collate_fn=TrainValCollateFn)


# test input with Arkit

In [ ]:
def print_cuda_memory_usage(device_id=0):
    t = torch.cuda.get_device_properties(device_id).total_memory
    r = torch.cuda.memory_reserved(device_id) 
    a = torch.cuda.memory_allocated(device_id)
    f = r - a  # free inside reserved

    print(f"CUDA Device ID: {device_id}")
    print(f"Total memory: {t / 1e9:.2f} GB")
    print(f"Reserved memory: {r / 1e9:.2f} GB")
    print(f"Allocated memory: {a / 1e9:.2f} GB")
    print(f"Free (inside reserved): {f / 1e9:.2f} GB")
data_iterator = iter(data_loader)

# 获取第一个元素
first_batch = next(data_iterator)
model(first_batch)

# Replace MLP

In [ ]:
from torch import nn
cls = nn.Sequential(
    nn.Linear(32, 128), 
    nn.BatchNorm1d(128), 
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(128, 512), 
    nn.BatchNorm1d(512), 
    nn.ReLU(inplace=True))
cls = cls.to(device)


#model.model.cls =cls

In [ ]:
print(num_ftrs)